<a href="https://colab.research.google.com/github/ptleskin/QuasiRandomCircle/blob/main/QuasiRandomCircleFill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quasi Random Circle Fill
This colab is a Python adaptation of my old code at [Pixelero blog](https://pixelero.wordpress.com/2014/11/26/spiral-moire/).

# Set up dependencies and imports

In [1]:
!pip install numpy scipy matplotlib networkx ipywidgets
%matplotlib inline

# this will allow the notebook to reload/refresh automatically within the runtime
%reload_ext autoreload
%autoreload 2

In [2]:
import ipywidgets
from ipywidgets import interact

import matplotlib.pyplot as plt
import numpy    as np
import networkx as nx
import re
from scipy.spatial import Delaunay

# Function definitions

In [38]:
def repeat(arr, N):
  ''' repeat the iterable arr until the length N '''
  for i in range(N):
    yield arr[i%len(arr)]


def formSpiral(N, f, pow=0.5):
  ''' spread N points along a spiral inside a unit circle '''
  # by default value pow=0.5 this is a square root
  theta = np.power(range(N),pow)
  radii = theta/max(theta)
  return radii*np.cos(f*theta), radii*np.sin(f*theta)


def drawSpiral(ax, X,Y, S=None, C=None):
  ''' render the spiral
  S: size by dot
  C: color by dot
  '''
  if S:
    sizes = np.array(list(repeat(S, len(X))))
    sizes = 3*sizes*sizes
  else:
    sizes = None

  if C:
    colors = list(repeat(C, len(X)))
  else:
    colors = None
  
  ax.scatter(X,Y, s=sizes, c=colors)

# Interactive rendering

In [39]:
S = [5-len(re.sub('0+$', '', '{0:04b}'.format(x))) for x in range(16)]
C = [6-len(re.sub('0+$', '', '{0:04b}'.format(x))) for x in range(16)]

def updateSpiral(f, N=1000, pow=0.5):
  fig = plt.figure(figsize=[12.0, 12.0])
  ax = fig.add_subplot(1, 1, 1)
  
  drawSpiral(ax, *formSpiral(N, f, pow), S=S, C='k')
  
  ax.axis("off")
  ax.set_aspect('equal')

  plt.show()
  return f

In [40]:
_ = ipywidgets.interact(updateSpiral, 
                      N=ipywidgets.IntSlider(min=100, max=10000, step=25, value=1000, continuous_update=False),
                      f=ipywidgets.FloatSlider(min=1.0, max=20.0, step=0.05, value=3.2, continuous_update=False),
                      pow=ipywidgets.FloatSlider(min=0.25, max=0.90, step=0.01, value=0.5, continuous_update=False)
                      )

interactive(children=(FloatSlider(value=3.2, continuous_update=False, description='f', max=20.0, min=1.0, step…

# Render the spiral as a minimum spanning tree

In [41]:
def graphSpiral(f, N=1000, pow=0.5):
  fig = plt.figure(figsize=[14.0, 14.0])
  ax = fig.add_subplot(111)

  # form the spiral and apply a Delaunay triangulation
  X,Y = formSpiral(N, f, pow)
  XY = np.array(list(zip(X,Y))) 
  tri = Delaunay(XY)

  # get the edges of the triangulation
  edges = list(tri.simplices[:,[0,1]])+list(tri.simplices[:,[1,2]])+list(tri.simplices[:,[2,0]])
  edges = set([(min(x), max(x)) for x in edges])
  dists = [(u,v, np.linalg.norm(XY[u]-XY[v])) for (u,v) in edges]

  # build a graph and extract the minimum spanning tree
  G = nx.Graph()
  G.add_weighted_edges_from(dists)
  T=nx.minimum_spanning_tree(G)

  # render the graph
  nx.drawing.draw_networkx(T,
                          pos=dict([(u, XY[u]) for u in T.nodes()]),
                          edge_color='grey',
                          with_labels=False,
                          node_size=3,
                          node_color='black'
                          )

  plt.show()  
  return f, N

In [42]:
_ = ipywidgets.interact(graphSpiral, 
                      N=ipywidgets.IntSlider(min=100, max=2500, step=10, value=750, continuous_update=False),
                      f=ipywidgets.FloatSlider(min=1.0, max=20.0, step=0.05, value=11.1, continuous_update=False),
                      pow=ipywidgets.FloatSlider(min=0.25, max=0.75, step=0.01, value=0.5, continuous_update=False)
                      )

interactive(children=(FloatSlider(value=11.1, continuous_update=False, description='f', max=20.0, min=1.0, ste…